In [1]:
import cv2
import numpy as np
from my_function import imshow

## 图像边缘检测

图像边缘检测是计算机视觉和图像处理中的一项基本任务，它用于识别图像中亮度变化明显的区域，这些区域通常对应于物体的边界

OpenCV提供了多种边缘检测算法，以下是一些常用的边缘检测函数及其适用场景：

|函数|算法|说明|适用场景|
|-----|-----|-----|-----|
|cv2.Canny()|Canny边缘检测|多阶段算法，检测效果较好，噪声抑制能力强|通用边缘检测，适合大多数场景|
|cv2.Sobel()|Sobel算子|基于一阶导数的边缘检测，可以检测水平和垂直边缘|检测水平和垂直边缘|
|cv2.Scharr()|Scharr算子|Sobel算子的改进版本，对边缘的响应更强|检测细微的边缘|
|cv2.Laplacian()|Laplacian算子|基于二阶导数的边缘检测，对噪声敏感|检测边缘和角点|

## 算子概念

在数学领域里，算子（operator）有别于物理的算符，是一种映射，一个向量空间的元素通过此映射（或模）在另一个向量空间（也有可能是相同的向量空间）中产生另一个元素

广义的算子就是框架中封装好的可重用的运算函数，如图像处理中常用的卷积算子、梯度算子等。在代码中,算子可能通过类(Class)或者函数(Function)来实现

## Prewitt算子

在一幅图像中，灰度值可以看作一个二维函数f(x,y)，在任意一点处，其梯度可以表示为一个向量：

$$\nabla f = \left( \frac{\partial f}{\partial x}, \frac{\partial f}{\partial y} \right)$$

我们知道，梯度的方向指向图像中灰度值增加最快的方向，其大小（模）表示了变化的剧烈程度，也就是边缘的强度

现在只考虑x方向的梯度变化，有梯度公式：

$$\frac{\partial f}{\partial x} = \lim_{\Delta x \to 0} \frac{f(x + \Delta x, y) - f(x, y)}{\Delta x}$$

因为图像中，像素是离散的，$\Delta x$最小取1，然后写为更准确的中心差分形式：

$$\frac{\partial f}{\partial x} \approx \frac{f(x + 1, y) - f(x - 1, y)}{2}$$

此处的$\frac{1}{2}$是一个缩放因子，只影响绝对值，不影响边缘检测的相对强弱，可以在最后统一归一化

所以，在一维上，我们假设有向量：$[p_{-1}, p_0, p_1]$，表示像素点在x方向的灰度值，那么$p_0$点的梯度可以表示为（是点积不是叉积）：

$$ [p_{-1}, p_0, p_1] \cdot \begin{bmatrix} -1 \\ 0 \\ 1 \end{bmatrix} = p_1 - p_{-1} $$

现在扩展至二维，假设有一个3x3的像素块：

$$\begin{bmatrix} p_{-1,-1} & p_{0,-1} & p_{1,-1} \\ p_{-1,0} & p_{0,0} & p_{1,0} \\ p_{-1,1} & p_{0,1} & p_{1,1} \end{bmatrix}$$

对x方向求导，$p_{0,0}$的x方向梯度可以表示为：

$$\begin{bmatrix} p_{-1,-1} & p_{0,-1} & p_{1,-1} \\ p_{-1,0} & p_{0,0} & p_{1,0} \\ p_{-1,1} & p_{0,1} & p_{1,1} \end{bmatrix} \cdot\begin{bmatrix} -1 & 0 & 1 \\ -1 & 0 & 1 \\ -1 & 0 & 1 \end{bmatrix}\\ = (p_{1,-1} + p_{1,0} + p_{1,1}) - (p_{-1,-1} + p_{-1,0} + p_{-1,1})$$

得到y方向边缘

在y方向同理可得：

$$\begin{bmatrix} p_{-1,-1} & p_{0,-1} & p_{1,-1} \\ p_{-1,0} & p_{0,0} & p_{1,0} \\ p_{-1,1} & p_{0,1} & p_{1,1} \end{bmatrix} \cdot\begin{bmatrix} -1 & -1 & -1 \\ 0 & 0 & 0 \\ 1 & 1 & 1 \end{bmatrix}\\ = (p_{-1,1} + p_{0,1} + p_{1,1}) - (p_{-1,-1} + p_{0,-1} + p_{1,-1})$$

得到x方向边缘

以上就是Prewitt算子的基本原理，其实质是通过卷积操作来计算图像在x和y方向的梯度，从而检测边缘

在实际操作时，算子以3x3的卷积核形式应用于图像，通过滑动窗口的方式遍历图像的每一个像素点，计算其梯度值，进而确定边缘位置

## Sobel算子

- Sobel 算子是一种基于梯度的边缘检测算子，它通过计算图像在水平和垂直方向上的梯度来检测边缘
- Sobel 算子结合了高斯平滑和微分操作，因此对噪声具有一定的抑制作用
- Sobel边缘检测算法比较简单，实际应用中效率比canny边缘检测效率要高

原理：

在上面的Prewitt算子的基础上，Sobel算子对中心像素行/列赋予了更高的权重，以增强边缘检测的效果，具体如下：

假设有矩阵：
$$\begin{bmatrix} p_{-1,-1} & p_{0,-1} & p_{1,-1} \\ p_{-1,0} & p_{0,0} & p_{1,0} \\ p_{-1,1} & p_{0,1} & p_{1,1} \end{bmatrix}$$

那么x方向的$p_{0,0}$的梯度可以表示为：

$$Gx=\begin{bmatrix} p_{-1,-1} & p_{0,-1} & p_{1,-1} \\ p_{-1,0} & p_{0,0} & p_{1,0} \\ p_{-1,1} & p_{0,1} & p_{1,1} \end{bmatrix} \cdot\begin{bmatrix} -1 & 0 & 1 \\ -2 & 0 & 2 \\ -1 & 0 & 1 \end{bmatrix} \\= (p_{1,-1} + 2p_{1,0} + p_{1,1}) - (p_{-1,-1} + 2p_{-1,0} + p_{-1,1})$$

在y方向同理可得：

$$Gy=\begin{bmatrix} p_{-1,-1} & p_{0,-1} & p_{1,-1} \\ p_{-1,0} & p_{0,0} & p_{1,0} \\ p_{-1,1} & p_{0,1} & p_{1,1} \end{bmatrix} \cdot\begin{bmatrix} -1 & -2 & -1 \\ 0 & 0 & 0 \\ 1 & 2 & 1 \end{bmatrix} \\= (p_{-1,1} + 2p_{0,1} + p_{1,1}) - (p_{-1,-1} + 2p_{0,-1} + p_{1,-1})$$

最终的梯度幅值可以通过以下公式计算：

$$G = \sqrt{Gx^2 + Gy^2}$$

函数原型：

`dst = cv2.Sobel(src, ddepth, dx, dy, ksize=3, scale=1, delta=0, borderType=cv2.BORDER_DEFAULT)`

参数说明：

- src：输入图像
- ddepth：输出图像的深度，通常使用 `cv2.CV_64F`，ddepth=-1 表示与输入图像相同的深度
- dx：x 方向上的导数阶数,检测出的是垂直方向上的边缘
- dy：y 方向上的导数阶数,检测出的是水平方向上的边缘
- ksize：Sobel 核的大小，默认为 3，可选值为 1、3、5、7
- scale：缩放因子，默认为 1，缩放因子用于调整梯度值的范围
- delta：可选的 delta 值，默认为 0
- borderType：边界填充类型，默认为 `cv2.BORDER_DEFAULT`

In [2]:
image=cv2.imread('./images/opencv_logo.jpg', cv2.IMREAD_GRAYSCALE)
# 计算 x 方向的梯度
sobel_x=cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
# 计算 y 方向的梯度
sobel_y=cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
# 计算梯度幅值
sobel=np.sqrt(sobel_x**2+sobel_y**2)
imshow(sobel_x=sobel_x, sobel_y=sobel_y, sobel_combined=sobel)

## Scharr算子

- Scharr算子是Sobel算子的改进版本，设计用于提高边缘检测的精度，特别是在处理细微边缘时表现更好
- Scharr算子通过调整卷积核的权重，使其对边缘的响应更强，从而提高了边缘检测的效果

原理：

Scharr算子使用以下两个3x3的卷积核来计算图像在水平和垂直方向上的梯度：

- 水平方向的卷积核：

$$Gx = \begin{bmatrix}
-3 & 0 & 3 \\
-10 & 0 & 10 \\
-3 & 0 & 3
\end{bmatrix} \times I$$

- 垂直方向的卷积核：

$$Gy = \begin{bmatrix}
-3 & -10 & -3 \\
0 & 0 & 0 \\
3 & 10 & 3
\end{bmatrix} \times I$$

最终的梯度幅值计算与Sobel算子相同

函数原型：

`dst = cv2.Scharr(src, ddepth, dx, dy, scale=1, delta=0, borderType=cv2.BORDER_DEFAULT)`

参数说明（scharr算子只支持ksize=3）：

- src：输入图像
- ddepth：输出图像的深度，通常使用 `cv2.CV_64F`
- dx：x 方向上的导数阶数
- dy：y 方向上的导数阶数
- scale：缩放因子，默认为 1
- delta：可选的 delta 值，默认为 0
- borderType：边界填充类型，默认为 `cv2.BORDER_DEFAULT`

In [3]:
scharr_x=cv2.Scharr(image,cv2.CV_64F,1,0,scale=1,delta=0,borderType=cv2.BORDER_DEFAULT)
scharr_y=cv2.Scharr(image,cv2.CV_64F,1,0,scale=1,delta=0,borderType=cv2.BORDER_DEFAULT)
scharr=np.sqrt(scharr_x**2+scharr_y**2)
imshow(scharr=scharr,scharr_x=scharr_x,scharr_y=scharr_y)

## Laplacian算子

- Laplacian算子是一种二阶微分算子，它通过计算图像的二阶导数来检测边缘
- Laplacian算子对噪声比较敏感，因此通常在使用之前会对图像进行高斯平滑处理

原理：

Laplacian算子使用以下3x3的卷积核来计算图像的二阶导数：

$$L = \begin{bmatrix}
0 & 1 & 0 \\
1 & -4 & 1 \\
0 & 1 & 0
\end{bmatrix} \times I$$

通过这个卷积核，可以得到图像的 Laplacian 值。Laplacian 值较大的区域通常对应于图像的边缘

函数原型：

`dst = cv2.Laplacian(src, ddepth, ksize=1, scale=1, delta=0, borderType=cv2.BORDER_DEFAULT)`

参数说明：

- src：输入图像
- ddepth：输出图像的深度，通常使用 cv2.CV_64F
- ksize：Laplacian 核的大小，默认为 1
- scale：缩放因子，默认为 1
- delta：可选的 delta 值，默认为 0
- borderType：边界填充类型，默认为 cv2.BORDER_DEFAULT

In [4]:
laplacian=cv2.Laplacian(image,cv2.CV_64F,ksize=3)
imshow(laplacian=laplacian)

## Canny边缘检测

- Canny 边缘检测是一种多阶段的边缘检测算法，由 John F. Canny 在 1986 年提出
- Canny 边缘检测被认为是边缘检测的"金标准"，因为它能够在噪声抑制和边缘定位之间取得良好的平衡

步骤：

1. **噪声抑制**：使用高斯滤波器对图像进行平滑处理，以减少噪声的影响
2. **计算梯度**：使用 Sobel 算子计算图像的梯度幅值和方向
3. **非极大值抑制**：沿着梯度方向，保留局部梯度最大的像素点，抑制其他像素点
4. **双阈值检测**：使用两个阈值（低阈值和高阈值）来确定真正的边缘。高于高阈值的像素点被认为是强边缘，低于低阈值的像素点被抑制，介于两者之间的像素点如果与强边缘相连则保留
5. **边缘连接**：通过滞后阈值处理，将弱边缘与强边缘连接起来，形成完整的边缘

函数原型：

`edges = cv2.Canny(image, threshold1, threshold2, apertureSize=3, L2gradient=False)`

参数说明：

- image：输入图像，必须是单通道的灰度图像
- threshold1：低阈值
- threshold2：高阈值
- apertureSize：Sobel 算子的孔径大小，默认为 3
- L2gradient：是否使用 L2 范数计算梯度幅值，默认为 False（使用 L1 范数）


In [5]:
canny=cv2.Canny(image,100,200,L2gradient=True)
imshow(canny=canny)

## 形态学梯度

- 形态学梯度是一种基于形态学操作的边缘检测方法，它通过计算图像的膨胀和腐蚀之间的差异来检测边缘
- 形态学梯度对于检测物体的边界和轮廓非常有效，特别是在处理二值图像时表现良好
- 详细解释见`图像形态学操作.ipynb`